# Setup

In [ ]:
# Image from Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import string
import re
import numpy as np
import pandas as pd
import tensorflow as tf
import keras.backend as K

import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')

import seaborn as sns # untuk visualisasi
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline

# potongan kode di bawah adalah agar gambar graph yang 
# dihasilkan mempunyai kualitas yang cukup baik
sns.set(rc={"figure.dpi":100, 'savefig.dpi':300})
sns.set_context('notebook')
sns.set_style("ticks")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
!pip install -q catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 9.7 MB/s eta 0:00:00


In [ ]:
import sklearn

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression

from xgboost import XGBRegressor, XGBClassifier # https://xgboost.readthedocs.io/en/stable/parameter.html
from sklearn.model_selection import cross_val_score

from catboost import CatBoostRegressor, CatBoostClassifier, Pool

from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification

from sklearn.svm import LinearSVC, SVC
from mlxtend.plotting import plot_decision_regions

from scipy.stats import ttest_rel
from sklearn.metrics import hamming_loss, accuracy_score, classification_report, mean_squared_error

In [ ]:
abusive_lexicon = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/abusive.csv')
dataset = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/re_dataset.csv', encoding='latin-1')

all_test_data = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/all_extracted_test_data.csv', encoding='latin-1')
all_synthetic_test_data = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/all_extracted_synthetic_test_data.csv', encoding='latin-1')
all_train_data = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/all_extracted_train_data.csv', encoding='latin-1')

kamus_alay = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/kamusalay.csv', encoding='latin-1')
potensi_bias = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/potensi_bias.csv', encoding='latin-1')
most_freq = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/most_freq_words.csv', encoding='latin-1')

# Prep

In [ ]:
top_n_hs = [
 ('cebong', 110.34811441960026),
 ('2019gantipresiden', 75.90069314600002),
 ('Exclamation', 70.55371595026722),
 ('jokowi', 63.371015645395964),
 ('prabowo', 54.86108861723573),
 ('ahok', 53.125796943099495),
 ('lengser', 42.777155871476005),
 ('Length', 36.84098374032803),
 ('Abusive', 35.469689057989),
 ('budaya', 34.90252416756176),
 ('tolol', 33.53126063723237),
 ('goblok', 33.463208660821344),
 ('Uppercase', 32.61247054509399),
 ('bubar', 29.583672984510986),
 ('dungu', 28.192095154682843),
 ('kampret', 25.453576436951316),
 ('bani', 25.453576436951316),
 ('dasar', 25.231513174062865),
 ('Words', 24.62759681765306),
 ('kafir', 24.564368023774918),
 ('otak', 23.521747205193947),
 ('katolik', 22.98481959802605),
 ('tipu', 21.29015625192861),
 ('bom', 21.069816411923135),
 ('biadab', 18.96557774364226),
 ('ekonomi', 18.55243737936092),
 ('kristen', 17.574428491215734),
 ('pki', 16.518813009230392),
 ('pilkada', 16.334821106469022),
 ('sontoloyo', 16.221324924819715),
 ('usir', 14.757229608394622),
 ('ngentot', 14.428125216807064),
 ('jawa', 14.428125216807064),
 ('cina', 14.316203993907742),
 ('yahudi', 13.962936525388983),
 ('hindu', 13.657753624378143),
 ('picek', 13.488824794728739),
 ('munafik', 12.658269845589082),
 ('nista', 12.563964219047698),
 ('rakyat', 12.379494305520401),
 ('asing', 12.36720412271323),
 ('sipit', 12.242350547041795),
 ('sby', 12.227681688589335),
 ('anies', 12.227681688589335),
 ('swt', 12.227681688589335),
 ('anies-sandi', 12.126958419731528),
 ('keparat', 12.126958419731528),
 ('pilih kepala daerah', 12.120723073700699),
 ('bangsat', 11.917097349474046),
 ('hancur', 11.917097349474046),
 ('tukang', 11.917097349474046),
 ('2018', 11.833752616444313),
 ('alam', 11.354058156167763),
 ('cewek', 11.081932759468454),
 ('kpk', 10.985124834688929),
 ('anjing', 10.558503723385709),
 ('kontol', 10.550765287814363),
 ('temu', 10.359741447128847),
 ('bacot', 10.259729735456526),
 ('kaum', 10.259729735456526),
 ('gubernur', 10.15230225715779),
 ('bajing', 9.8784526034065),
 ('tingkat', 9.824414210128495),
 ('korupsi', 9.365197784332462),
 ('komunis', 9.306022442876644),
 ('wkwk', 9.293863998507947),
 ('hina', 9.212676473902215),
 ('cocot', 9.166008983142897),
 ('memek', 9.135252718767692),
 ('rezim', 8.588506539575318),
 ('bapak', 8.566477414978369),
 ('tangkap', 8.55629534273258),
 ('cacat', 8.184967172438588),
 ('sandi', 8.114389642031412),
 ('myanmar', 8.058781892287833),
 ('bong', 8.058781892287833),
 ('umat', 8.003339444735863),
 ('anjir', 7.8687842013719616),
 ('All Caps', 7.78898551574197),
 ('buaya', 7.539118818324024),
 ('bego', 7.479306439820447),
 ('kayak', 7.362203594452939),
 ('bisu', 7.339305705539183),
 ('titit', 7.339305705539183),
 ('bangun', 7.0629399325201705),
 ('tka', 6.835304245647373),
 ('an', 6.6015073588477176),
 ('dpr', 6.424447096363063),
 ('teriak', 6.424447096363063),
 ('congor', 6.226864258327995),
 ('berani', 6.226864258327995),
 ('mu', 6.226864258327995),
 ('citra', 6.226864258327995),
 ('masyarakat', 6.146039642845457),
 ('babi', 6.060842695503713),
 ('bangkai', 6.059474231273955),
 ('bahasa', 6.059474231273955),
 ('islam', 6.0089486417564935),
 ('ngewe', 5.868650990363385),
 ('bodoh', 5.846977175607727),
 ('bela', 5.756227882921352),
 ('sekolah', 5.725593417401544),
 ('dengar', 5.725593417401544),
 ('antek', 5.6963070161213505),
 ('geblek', 5.683811250175008),
 ('teman', 5.3827815686689195),
 ('rohingya', 5.361090365224104),
 ('kunti', 5.264757854231538),
 ('bejat', 5.113139692929426),
 ('banyak', 5.113139692929426),
 ('lu', 5.113139692929426),
 ('idiot', 4.896896702021349),
 ('cinta', 4.896896702021349),
 ('takut', 4.896896702021349),
 ('nusantara', 4.793343394228391),
 ('hal', 4.68854049625912),
 ('arab', 4.68854049625912),
 ('lanjut', 4.68854049625912),
 ('lesbi', 4.509053852616743),
 ('edan', 4.436132363569043),
 ('tulis', 4.436132363569043),
 ('pintar', 4.433757799363106),
 ('lupa', 4.422225417042576),
 ('kota', 4.422225417042576),
 ('sayang', 4.2942534796581455),
 ('onta', 4.223504369221073),
 ('malu', 4.222349848962606),
 ('habis', 4.219757859048472),
 ('pilih', 4.205384807041673),
 ('lo', 4.044727738973866),
 ('celeng', 4.038666534685073),
 ('lahir', 4.021626043877695),
 ('bukti', 3.9395050335731843),
 ('jalan', 3.8143983944773048),
 ('aman', 3.8143983944773048),
 ('pimpin', 3.7715922200041265),
 ('autis', 3.754553201322991),
 ('gendut', 3.754553201322991),
 ('buat', 3.730906008609057),
 ('janji', 3.730906008609057),
 ('nete', 3.7121295685606563),
 ('lgbt', 3.7121295685606563),
 ('beli', 3.70905155524417),
 ('dongok', 3.3999136212368546),
 ('cerita', 3.370797456168331),
 ('tuhan', 3.3563032593346906),
 ('banci', 3.2422704924007655),
 ('polisi', 3.2422704924007655),
 ('kelas', 3.211761064134776),
 ('warga', 3.185111161805482),
 ('bangsa', 3.158497094728439),
 ('aku', 3.158497094728439),
 ('buta', 3.143997578267667),
 ('monyet', 3.12851638683714),
 ('sok', 3.0971072182810566),
 ('anti', 3.0485926341658467),
 ('jamban', 3.0145439139170023),
 ('ngaceng', 3.0012530283924757),
 ('ajar', 2.8354652386848658),
 ('bunuh', 2.8053962792153384),
 ('tunggu', 2.8053962792153384),
 ('selesai', 2.74011246528556),
 ('berak', 2.674854003880392),
 ('cupu', 2.674854003880392),
 ('china', 2.654458990311588),
 ('lawan', 2.654458990311588),
 ('kepala', 2.6320554617381013),
 ('kenal', 2.6320554617381013),
 ('tolak', 2.61609664297876),
 ('rumah', 2.61609664297876),
 ('biar', 2.584697221063741),
 ('rusak', 2.405743271678149),
 ('setan', 2.3799380055695485),
 ('2019', 2.3799380055695485),
 ('mampus', 2.3491965549582714),
 ('gembel', 2.334312505136287),
 ('gay', 2.24915047099921),
 ('sange', 2.24915047099921),
 ('cari', 2.141160467753218),
 ('beda', 2.141160467753218),
 ('hilang', 2.1352542609604095),
 ('baca', 2.0885532975774357),
 ('apa', 2.0885532975774357),
 ('orang', 2.066388543764246),
 ('eh', 2.039051064472662),
 ('fitnah', 1.9581172452955873),
 ('marah', 1.8729992802532807),
 ('bang', 1.8222672481800017),
 ('surat beri tahun', 1.8222672481800017),
 ('senang', 1.793810836870813),
 ('dunia', 1.7726967093853232),
 ('kitab', 1.7558984243824687),
 ('modar', 1.6955566211412976),
 ('silit', 1.6795116844114732),
 ('ateis', 1.6795116844114732),
 ('uang', 1.6751292017774027),
 ('harga', 1.6751292017774027),
 ('mulu', 1.6751292017774027),
 ('bantu', 1.6076494981312992),
 ('no', 1.6076494981312992),
 ('taplak', 1.56443525902955),
 ('hak', 1.56443525902955),
 ('jembut', 1.498242675412765),
 ('gembrot', 1.498242675412765),
 ('lonte', 1.498242675412765),
 ('presiden', 1.4697889678022833),
 ('muslim', 1.4102638994707037),
 ('pantat', 1.39797830756791),
 ('partai', 1.3747898526749565),
 ('bawa', 1.35272484383147),
 ('bispak', 1.3360087336454554),
 ('cungkring', 1.3360087336454554),
 ('koreng', 1.3360087336454554),
 ('seks', 1.3360087336454554),
 ('serbet', 1.3360087336454554),
 ('dukung', 1.3352932676981297),
 ('negeri', 1.2241769021605318),
 ('gaji', 1.2117419020199114),
 ('langsung', 1.206872213558489),
 ('partai komunis indonesia', 1.2033748869227199),
 ('berengsek', 1.1883265039447972),
 ('muka', 1.1883265039447972),
 ('kalah', 1.1868363835032474),
 ('tak', 1.1817908542308877),
 ('butuh', 1.181462898836725),
 ('in', 1.181462898836725),
 ('ku', 1.0898461126904577),
 ('kejam', 1.0646835610936336),
 ('bandara', 1.0646835610936336),
 ('agama', 1.0558361648393257),
 ('culun', 1.0399793227694663),
 ('haha', 1.0360407229757316),
 ('amp', 1.0149578538074995),
 ('pikir', 1.002590258794752),
 ('ajak', 0.987238496113362),
 ('ganti', 0.9529182771085576),
 ('nama', 0.9470447854929663),
 ('laku', 0.9119539461189773),
 ('fiksi', 0.8795133448239209),
 ('enak', 0.8795133448239209),
 ('indonesia', 0.863349459687896),
 ('allah', 0.7782263781933868),
 ('budek', 0.7741723925292983),
 ('wakil', 0.7741723925292983),
 ('ambil', 0.7741723925292983),
 ('iman', 0.7741723925292983),
 ('tuyul', 0.7485267969621711),
 ('sitip', 0.7485267969621711),
 ('panasbung', 0.7485267969621711),
 ('sompret', 0.6978151412973321),
 ('akun', 0.6824723337284011),
 ('bayar', 0.6824723337284011),
 ('rejim', 0.6730734564921673),
 ('benar', 0.6730734564921673),
 ('buang', 0.6730734564921673),
 ('bahas', 0.6730734564921673),
 ('nyata', 0.6655411212446929),
 ('coba', 0.6655411212446929),
 ('hoaks', 0.6655411212446929),
 ('terima', 0.6655411212446929),
 ('isi', 0.6499351209575555),
 ('kampung', 0.6499351209575555),
 ('lihat', 0.6499198837711107),
 ('jancuk', 0.604327391204703),
 ('kampang', 0.604327391204703),
 ('udik', 0.604327391204703),
 ('buddha', 0.5990572251404706),
 ('jijik', 0.5886917794420856),
 ('jablay', 0.5812616064601234),
 ('kena', 0.553829773949056),
 ('milik', 0.553829773949056),
 ('kemarin', 0.553829773949056),
 ('dan lain lain', 0.553829773949056),
 ('ga', 0.553829773949056),
 ('pakai', 0.5219258990602881),
 ('ampas', 0.5189219477713409),
 ('sinting', 0.5189219477713409),
 ('transgender', 0.5189219477713409),
 ('maju', 0.5027169459407322),
 ('besar', 0.46877063948455655),
 ('makan', 0.4664957666730979),
 ('nih', 0.45815360382787496),
 ('harap', 0.4424353455058476),
 ('berita', 0.4424353455058476),
 ('anggap', 0.4424353455058476),
 ('ah', 0.3951764468134211),
 ('susah', 0.3951764468134211),
 ('suka', 0.37998408079499013),
 ('masuk', 0.373930237684931),
 ('sadar', 0.3398812201953986),
 ('tertawa', 0.3313564892037199),
 ('ulama', 0.3194533384033239),
 ('gerak', 0.2971188304577722),
 ('jadi', 0.28966689141361457),
 ('jual', 0.28966689141361457),
 ('tinggal', 0.2794069974658108),
 ('tuju', 0.27874429576307563),
 ('deh', 0.2777907241189218),
 ('kuasa', 0.2777907241189218),
 ('tau', 0.2743052932853056),
 ('ri', 0.2544360939881155),
 ('buka', 0.23849790514891414),
 ('bokong', 0.22059186595079452),
 ('paham', 0.21878433642930203),
 ('mati', 0.2118327884389553),
 ('menang', 0.18758952544075416),
 ('bangga', 0.18758952544075416),
 ('kuat', 0.1855826743982132),
 ('percaya', 0.1855826743982132),
 ('keluarga', 0.1855826743982132),
 ('utang', 0.16930869228961004),
 ('gak', 0.16239308309221592),
 ('politik', 0.1513973408094525),
 ('hidup', 0.14389847150680732),
 ('bencong', 0.12686358484234905),
 ('pecun', 0.12686358484234905),
 ('ayo', 0.1261078623242918),
 ('bilang', 0.1222699622459239),
 ('jakarta', 0.11397400809697021),
 ('anak', 0.11159592849377392),
 ('noob', 0.11014014422085706),
 ('homo', 0.11014014422085706),
 ('gila', 0.10099417605764564),
 ('tonton', 0.10099417605764564),
 ('proyek', 0.10099417605764564),
 ('paksa', 0.10099417605764564),
 ('salah', 0.0987762823165351),
 ('hati', 0.09812781599452242),
 ('usaha', 0.09244954520887085),
 ('moga', 0.09158628995731974),
 ('kerja', 0.08900367696055259),
 ('manusia', 0.08158501421353827),
 ('kalo', 0.08158501421353827),
 ('perintah', 0.0796049062328596),
 ('main', 0.0792196409830799),
 ('suci', 0.06511816992696887),
 ('selamat', 0.06511816992696887),
 ('bicara', 0.0633668407978869),
 ('enggak', 0.06333438176032172),
 ('urus', 0.059605492760204944),
 ('sakit', 0.059605492760204944),
 ('cepat', 0.059605492760204944),
 ('aduh', 0.059605492760204944),
 ('sampah', 0.048726761186433),
 ('kacrut', 0.042209457355024425),
 ('bokep', 0.042209457355024425),
 ('iblis', 0.0324102708147798),
 ('nabi', 0.03150796673508067),
 ('lucu', 0.03150796673508067),
 ('bau', 0.03150796673508067),
 ('hukum', 0.02973337224992938),
 ('negara', 0.026217724889937923),
 ('bikin', 0.02403027808666823),
 ('turun', 0.019740585586590788),
 ('diam', 0.019740585586590788),
 ('kasih', 0.01958380114471903),
 ('ikut', 0.018696306194827452),
 ('alay', 0.016168186351685133),
 ('burik', 0.016168186351685133),
 ('perek', 0.016168186351685133),
 ('tai', 0.014568551917638136),
 ('sarap', 0.009829796551859917),
 ('om', 0.009829796551859917),
 ('menteri', 0.009829796551859917),
 ('najis', 0.0064564541069301136),
 ('benci', 0.0064564541069301136),
 ('foto', 0.0064564541069301136),
 ('calon', 0.005789203298415481),
 ('bro', 0.005789203298415481),
 ('kunyuk', 0.003903982385145879),
 ('isu', 0.003903982385145879),
 ('mata', 2.9849408712603493e-05),
 ('hasil', 1.7794839710889455e-05),
 ('malam', 1.1825324761078608e-05)
]

In [ ]:
top_n_a = [
 ('Abusive', 863.4908194062502),
 ('cebong', 148.4593090814069),
 ('Length', 79.29269942971099),
 ('Words', 59.50479688978978),
 ('goblok', 55.09732272941384),
 ('kampret', 50.05443759533496),
 ('tolol', 50.05443759533496),
 ('anjir', 46.712693784996986),
 ('kontol', 41.73005782855407),
 ('bajing', 41.73005782855407),
 ('anjing', 41.47634233502745),
 ('gubernur', 38.46244074632886),
 ('dungu', 38.42812007858035),
 ('memek', 38.42812007858035),
 ('jokowi', 37.646464279376865),
 ('otak', 32.920592046427224),
 ('kristen', 31.533762133903572),
 ('banci', 30.241821267850764),
 ('kunyuk', 30.241821267850764),
 ('ngentot', 30.241821267850764),
 ('bodoh', 28.95780496339941),
 ('presiden', 27.98420451854762),
 ('asing', 27.932181731484942),
 ('setan', 27.239719564696824),
 ('babi', 26.77481003150012),
 ('bani', 26.77481003150012),
 ('budaya', 26.51965566357033),
 ('berengsek', 25.37648906686394),
 ('bacot', 25.183904543250843),
 ('bego', 25.183904543250843),
 ('indonesia', 25.119144267263906),
 ('dasar', 24.102124285459052),
 ('bangsat', 23.76237059168414),
 ('agama', 22.985496135337815),
 ('kafir', 22.734112683990233),
 ('ekonomi', 22.43493229360794),
 ('titit', 22.152058999700508),
 ('edan', 21.982501378665294),
 ('tai', 21.60724658305677),
 ('bom', 21.021342867174837),
 ('congor', 20.545540839028977),
 ('Exclamation', 20.110055391749285),
 ('pki', 19.799716267517574),
 ('katolik', 19.677489883996248),
 ('sontoloyo', 18.942802721088434),
 ('bau', 17.343831320228837),
 ('biadab', 17.222731900406664),
 ('bubar', 16.335700694035832),
 ('yahudi', 15.905872147995746),
 ('cocot', 15.748613373361865),
 ('geblek', 15.748613373361865),
 ('picek', 15.748613373361865),
 ('2018', 15.670383149469032),
 ('kayak', 15.489002983851245),
 ('anti', 15.196579396494613),
 ('onta', 15.048342416518077),
 ('keparat', 14.157135679594232),
 ('sarap', 14.157135679594232),
 ('lengser', 14.147086068358899),
 ('islam', 14.117573054248526),
 ('ngewe', 14.083010366229777),
 ('pilkada', 13.500774578528109),
 ('dongok', 12.569385099863522),
 ('sipit', 12.099171450237005),
 ('hindu', 11.6995671929149),
 ('negara', 11.460435584614928),
 ('gembel', 10.985348556576627),
 ('mampus', 10.985348556576627),
 ('rakyat', 10.953172063695),
 ('2019', 10.939397078071806),
 ('haha', 10.740008201573204),
 ('wkwk', 10.41863026232282),
 ('pilih kepala daerah', 10.383941408766471),
 ('bejat', 9.953738304708134),
 ('najis', 9.953738304708134),
 ('lu', 9.953738304708134),
 ('tolak', 9.887842777595237),
 ('nusantara', 9.675415246250092),
 ('nete', 9.442861914159925),
 ('bencong', 9.405013033250682),
 ('pecun', 9.405013033250682),
 ('silit', 9.405013033250682),
 ('bokong', 9.405013033250682),
 ('pantat', 9.405013033250682),
 ('bong', 9.405013033250682),
 ('All Caps', 9.401864501623932),
 ('munafik', 9.341456579443827),
 ('jawa', 9.24206851749332),
 ('rezim', 9.069748861856723),
 ('masyarakat', 9.048407879292201),
 ('nih', 8.658359300624358),
 ('temu', 8.425143161492768),
 ('swt', 8.38969486146485),
 ('mulu', 8.38969486146485),
 ('dukung', 8.044588884931862),
 ('kpk', 7.957374284792215),
 ('mu', 7.880924916472622),
 ('jancuk', 7.828365574156539),
 ('burik', 7.828365574156539),
 ('kampang', 7.828365574156539),
 ('perek', 7.828365574156539),
 ('bangun', 7.806011928100646),
 ('sandi', 7.764551229958548),
 ('ri', 7.764551229958548),
 ('prabowo', 7.542974037014236),
 ('dunia', 7.529594056244415),
 ('antek', 7.473258503901397),
 ('tukang', 7.454858752648001),
 ('politik', 7.3477026537570795),
 ('bahasa', 7.3189357639820525),
 ('aman', 7.3189357639820525),
 ('komunis', 7.2148981781421),
 ('negeri', 7.191483666960961),
 ('warga', 6.781417769485377),
 ('alam', 6.6835319479262045),
 ('2019gantipresiden', 6.531320610058052),
 ('buaya', 6.460751242232326),
 ('jablay', 6.4232113894554494),
 ('jijik', 6.4232113894554494),
 ('pilih', 6.414994720663546),
 ('modar', 6.255393283964713),
 ('sinting', 6.255393283964713),
 ('ngaceng', 6.255393283964713),
 ('pimpin', 6.222894097939597),
 ('in', 6.198923291530308),
 ('beda', 6.1641321861986915),
 ('gerak', 6.05160028978836),
 ('sby', 5.978638407370967),
 ('dpr', 5.978638407370967),
 ('partai komunis indonesia', 5.978638407370967),
 ('pintar', 5.964198718963567),
 ('buddha', 5.8103172903344245),
 ('wakil', 5.423713552021778),
 ('tingkat', 5.423713552021778),
 ('paksa', 5.423713552021778),
 ('lo', 5.260234034462659),
 ('ulama', 5.057212415358934),
 ('budek', 4.995257766357115),
 ('isi', 4.995257766357115),
 ('aduh', 4.995257766357115),
 ('tau', 4.969633646905201),
 ('celeng', 4.952071788546622),
 ('hukum', 4.865965843764239),
 ('percaya', 4.7929198970337685),
 ('tka', 4.7929198970337685),
 ('sompret', 4.686083327393821),
 ('sange', 4.686083327393821),
 ('cewek', 4.667022767297652),
 ('kaum', 4.667022767297652),
 ('sok', 4.6346199975051565),
 ('kunti', 4.5123704971010365),
 ('tuju', 4.3757833884206745),
 ('kelas', 4.213129496637429),
 ('iblis', 4.033879466249808),
 ('cina', 4.016213674833364),
 ('kuasa', 3.874735941261297),
 ('myanmar', 3.8648532861162415),
 ('ateis', 3.8648532861162415),
 ('banyak', 3.856030819276307),
 ('anies', 3.836570717378089),
 ('coba', 3.836570717378089),
 ('enak', 3.711472844308974),
 ('sayang', 3.7080194193383145),
 ('tak', 3.657994379669277),
 ('china', 3.644260024794952),
 ('bisu', 3.644260024794952),
 ('jual', 3.644260024794952),
 ('hal', 3.644260024794952),
 ('arab', 3.644260024794952),
 ('lawan', 3.644260024794952),
 ('lanjut', 3.644260024794952),
 ('kitab', 3.5805331862474716),
 ('rejim', 3.573501072211223),
 ('buang', 3.573501072211223),
 ('lupa', 3.573501072211223),
 ('kota', 3.573501072211223),
 ('bahas', 3.573501072211223),
 ('udik', 3.524405379433369),
 ('amp', 3.4003639815090283),
 ('ajar', 3.3646192345426162),
 ('kerja', 3.3234777035061085),
 ('autis', 3.218303963339197),
 ('gendut', 3.218303963339197),
 ('kejam', 3.218303963339197),
 ('berak', 3.120422928861467),
 ('cupu', 3.120422928861467),
 ('jembut', 3.120422928861467),
 ('kacrut', 3.120422928861467),
 ('bokep', 3.120422928861467),
 ('lonte', 3.120422928861467),
 ('usir', 3.0915979319808975),
 ('maju', 3.0915979319808975),
 ('selamat', 3.0915979319808975),
 ('korupsi', 3.0886060041836245),
 ('lahir', 3.0886060041836245),
 ('proyek', 3.0886060041836245),
 ('anies-sandi', 2.973017182488492),
 ('lgbt', 2.973017182488492),
 ('idiot', 2.9022564917864293),
 ('tipu', 2.846491818133174),
 ('gak', 2.846491818133174),
 ('fiksi', 2.802227822652549),
 ('apa', 2.802227822652549),
 ('calon', 2.802227822652549),
 ('ah', 2.802081931762181),
 ('gila', 2.7679277531841624),
 ('lihat', 2.684056035567429),
 ('isu', 2.6203390923645267),
 ('rohingya', 2.5727203602376973),
 ('jamban', 2.385129617903958),
 ('usaha', 2.385129617903958),
 ('hidup', 2.3660247217303243),
 ('umat', 2.331483576893449),
 ('dan lain lain', 2.3125877125533827),
 ('no', 2.3125877125533827),
 ('buta', 2.169543233365026),
 ('turun', 2.169543233365026),
 ('nyata', 2.169543233365026),
 ('jakarta', 2.169543233365026),
 ('uang', 2.169543233365026),
 ('hoaks', 2.169543233365026),
 ('harga', 2.169543233365026),
 ('diam', 2.065737982624399),
 ('malu', 2.065737982624399),
 ('teriak', 2.065737982624399),
 ('fitnah', 2.0303011691301758),
 ('tinggal', 2.023987801305843),
 ('bangkai', 1.9925051047542262),
 ('bro', 1.9925051047542262),
 ('gay', 1.9281003147527664),
 ('foto', 1.8758027542996993),
 ('jadi', 1.8462717231092483),
 ('kenal', 1.8462717231092483),
 ('tulis', 1.7426542762940196),
 ('susah', 1.7426542762940196),
 ('bikin', 1.7251244393491723),
 ('bawa', 1.6833086867994533),
 ('rumah', 1.6833086867994533),
 ('hina', 1.6700469296292266),
 ('makan', 1.6532605272385101),
 ('muslim', 1.6402666145601208),
 ('monyet', 1.5719651123710912),
 ('bispak', 1.5583993721375502),
 ('cungkring', 1.5583993721375502),
 ('koreng', 1.5583993721375502),
 ('seks', 1.5583993721375502),
 ('serbet', 1.5583993721375502),
 ('hilang', 1.537672733422443),
 ('eh', 1.519227339993927),
 ('cepat', 1.4084436326813936),
 ('citra', 1.4084436326813936),
 ('iman', 1.4084436326813936),
 ('nama', 1.4053281999490739),
 ('jalan', 1.3439793089736656),
 ('kuat', 1.3439793089736656),
 ('nista', 1.3141244150490015),
 ('sekolah', 1.3136535397120097),
 ('akun', 1.2858522676684145),
 ('surat beri tahun', 1.2858522676684145),
 ('lesbi', 1.270399174875125),
 ('an', 1.1949732879671218),
 ('buat', 1.1949732879671218),
 ('sakit', 1.1949732879671218),
 ('kampung', 1.1949732879671218),
 ('cari', 1.0866471226713792),
 ('taplak', 1.0797919556584632),
 ('hak', 1.0797919556584632),
 ('menteri', 1.0797919556584632),
 ('besar', 1.0556744957007853),
 ('ganti', 1.0402828868129466),
 ('harap', 1.0059916624036236),
 ('kalah', 1.0059916624036236),
 ('cinta', 0.9852990075736564),
 ('hasil', 0.9852990075736564),
 ('moga', 0.9789768879847642),
 ('bantu', 0.9789768879847642),
 ('milik', 0.9789768879847642),
 ('homo', 0.9588072146514374),
 ('manusia', 0.8421000218382517),
 ('paham', 0.8384266567750978),
 ('muka', 0.8048116066133647),
 ('bandara', 0.7668207863592372),
 ('suka', 0.7418146677712607),
 ('bapak', 0.700449113459243),
 ('suci', 0.700449113459243),
 ('Uppercase', 0.6907540260704155),
 ('utang', 0.6693378048475362),
 ('nabi', 0.6483135288160334),
 ('selesai', 0.6483135288160334),
 ('menang', 0.6483135288160334),
 ('aku', 0.6469058652437571),
 ('orang', 0.6461942931003359),
 ('tuyul', 0.6417422806181589),
 ('sitip', 0.6417422806181589),
 ('panasbung', 0.6417422806181589),
 ('tunggu', 0.602452407633113),
 ('kalo', 0.5339412555653444),
 ('cacat', 0.509544417818766),
 ('gaji', 0.5015632655562706),
 ('tuhan', 0.45625172717915397),
 ('ikut', 0.45625172717915397),
 ('beli', 0.4541365179216576),
 ('baca', 0.41606943005305763),
 ('urus', 0.38024468242129716),
 ('ambil', 0.38024468242129716),
 ('bukti', 0.37741663854684154),
 ('sampah', 0.36356575700884786),
 ('bang', 0.36356575700884786),
 ('bela', 0.36356575700884786),
 ('bayar', 0.36356575700884786),
 ('benar', 0.3485733333409302),
 ('mati', 0.3485733333409302),
 ('bilang', 0.3449206940031362),
 ('transgender', 0.3340098566245529),
 ('allah', 0.2979843576770446),
 ('bicara', 0.29553906173996985),
 ('bangsa', 0.2951640803187311),
 ('takut', 0.2951640803187311),
 ('perintah', 0.28640736946711703),
 ('berani', 0.2736038896535338),
 ('marah', 0.2736038896535338),
 ('deh', 0.25551689677859507),
 ('main', 0.24737458058895198),
 ('hati', 0.22948889372691048),
 ('terima', 0.21810315580649647),
 ('tangkap', 0.2102457619314634),
 ('kemarin', 0.2102457619314634),
 ('salah', 0.20820939155966392),
 ('ampas', 0.20005101751840557),
 ('ku', 0.1877460344442704),
 ('berita', 0.16796329492833492),
 ('benci', 0.16796329492833492),
 ('cerita', 0.16796329492833492),
 ('anggap', 0.16796329492833492),
 ('sadar', 0.1466795322692125),
 ('om', 0.10859924736042406),
 ('tertawa', 0.10859924736042406),
 ('kasih', 0.10172459101341341),
 ('teman', 0.10150875326518632),
 ('gembrot', 0.09993015893759158),
 ('ahok', 0.09509959515183121),
 ('butuh', 0.0837702044689613),
 ('malam', 0.0837702044689613),
 ('partai', 0.0751303802523778),
 ('kepala', 0.06749051264880133),
 ('mata', 0.05675284380603185),
 ('pakai', 0.05224747136015181),
 ('noob', 0.04181680467569487),
 ('polisi', 0.040743551035547304),
 ('dengar', 0.040743551035547304),
 ('ajak', 0.040743551035547304),
 ('lucu', 0.03455609835155749),
 ('bangga', 0.03455609835155749),
 ('bunuh', 0.03353607065325695),
 ('masuk', 0.03198005500250606),
 ('senang', 0.031206366109533316),
 ('ayo', 0.031206366109533316),
 ('pikir', 0.031142518647085896),
 ('laku', 0.031142518647085896),
 ('enggak', 0.03075478404573651),
 ('rusak', 0.03075478404573651),
 ('keluarga', 0.017160956428000983),
 ('anak', 0.012739103178170884),
 ('langsung', 0.00872924023815085),
 ('culun', 0.008549200449532182),
 ('hancur', 0.00521362009616307),
 ('kena', 0.00521362009616307),
 ('ga', 0.00521362009616307),
 ('biar', 0.004950801659732137),
 ('janji', 0.0021877862956420943),
 ('tonton', 0.0021877862956420943),
 ('alay', 0.0017299712053637664),
 ('habis', 1.3688499392693038e-05),
 ('buka', 1.3688499392693038e-05)
]

In [ ]:
hs_feat, hs_val = zip(* top_n_hs)
hs_top_feats = list(hs_feat)

a_feat, a_val = zip(* top_n_a)
a_top_feats = list(a_feat)

num_feats = 360

In [ ]:
# HS Features
HS_features = hs_top_feats[:num_feats]

In [ ]:
# A Features
A_features = a_top_feats[:num_feats]

In [ ]:
features = list(set(HS_features + A_features))

In [ ]:
train_data = all_train_data[all_train_data.columns.intersection(features)]
test_data = all_test_data[all_test_data.columns.intersection(features)]
synthetic_test_data = all_synthetic_test_data[all_synthetic_test_data.columns.intersection(features)]

In [ ]:
def get_x_y(df, label):
  x = df.loc[:, df.columns != 'Tweet']
  

  if label == 'HS':
    x = x[x.columns.intersection(HS_features)]
    y = df.loc[:, df.columns == 'HS']
  elif label == 'A':
    x = x[x.columns.intersection(A_features)]
    y = df.loc[:, df.columns == 'A']
  elif label == 'all HS':
    x = x[x.columns.intersection(features)]
    y = df.loc[:, df.columns == 'HS']
  else:
    x = x[x.columns.intersection(features)]
    y = df.loc[:, df.columns == 'A']

  return x, y

In [ ]:
# train
X_train = train_data.loc[:, train_data.columns != 'Tweet']
X_train = X_train.loc[:, X_train.columns != 'HS']
X_train = X_train.loc[:, X_train.columns != 'A']
X_train = X_train[X_train.columns.intersection(features)]

X_train_HS = X_train[X_train.columns.intersection(HS_features)]
Y_train_HS = all_train_data.loc[:, all_train_data.columns == 'HS']
X_train_A = X_train[X_train.columns.intersection(A_features)]
Y_train_A = all_train_data.loc[:, all_train_data.columns == 'A']

# test
X_test = test_data.loc[:, test_data.columns != 'Tweet']
X_test = X_test.loc[:, X_test.columns != 'HS']
X_test = X_test.loc[:, X_test.columns != 'A']
X_test = X_test[X_test.columns.intersection(features)]

X_test_HS = X_test[X_test.columns.intersection(HS_features)]
Y_test_HS = all_test_data.loc[:, all_test_data.columns == 'HS']
X_test_A = X_test[X_test.columns.intersection(A_features)]
Y_test_A = all_test_data.loc[:, all_test_data.columns == 'A']

# syntest
X_stest = synthetic_test_data.loc[:, synthetic_test_data.columns != 'Tweet']
X_stest = X_stest.loc[:, X_stest.columns != 'HS']
X_stest = X_stest.loc[:, X_stest.columns != 'A']
X_stest = X_stest[X_stest.columns.intersection(features)]

X_stest_HS = X_stest[X_stest.columns.intersection(HS_features)]
Y_stest_HS = all_synthetic_test_data.loc[:, all_synthetic_test_data.columns == 'HS']
X_stest_A = X_stest[X_stest.columns.intersection(A_features)]
Y_stest_A = all_synthetic_test_data.loc[:, all_synthetic_test_data.columns == 'A']

In [ ]:
X_train

Exclamation  Abusive  Length  Words  Uppercase  All Caps  jokowi  islam  \
0               1        1      74      6          2         0       0      0   
1               0        0      77      6          4         0       0      1   
2               0        1     135      9          6         0       1      0   
3               0        1     133     11          4         0       0      0   
4               0        1     174     13          2         0       0      0   
...           ...      ...     ...    ...        ...       ...     ...    ...   
9214            0        1      49      6          4         0       0      0   
9215            1        0     118     10          3         0       0      0   
9216            0        1      52      6          5         0       0      0   
9217            0        1      61      5         49         1       0      1   
9218            0        1     232     22         12         0       0      0   

      indonesia  prabowo  ...  foto  proyek  marah  iman  bangga  muka  paksa  \
0             0        0  ...     0       0      0     0       0     0      0   
1             0        0  ...     0       0      0     0       0     0      0   
2             0        1  ...     0       0      0     0       0     0      0   
3             0        0  ...     0       0      0     0       0     0      0   
4             0        0  ...     0       0      0     0       0     0      0   
...         ...      ...  ...   ...     ...    ...   ...     ...   ...    ...   
9214          0        0  ...     0       0      0     0       0     0      0   
9215          0        0  ...     0       0      0     0       0     0      0   
9216          0        0  ...     0       0      0     0       0     0      0   
9217          0        0  ...     0       0      0     0       0     0      0   
9218          0        0  ...     0       0      0     0       0     0      0   

      lanjut  bandara  menteri  
0          0        0        0  
1          0        0        0  
2          0        0        0  
3          0        0        0  
4          0        0        0  
...      ...      ...      ...  
9214       0        0        0  
9215       0        0        0  
9216       0        0        0  
9217       0        0        0  
9218       0        0        0  

[9219 rows x 374 columns]

In [ ]:
X_train_HS

Exclamation  Abusive  Length  Words  Uppercase  All Caps  jokowi  islam  \
0               1        1      74      6          2         0       0      0   
1               0        0      77      6          4         0       0      1   
2               0        1     135      9          6         0       1      0   
3               0        1     133     11          4         0       0      0   
4               0        1     174     13          2         0       0      0   
...           ...      ...     ...    ...        ...       ...     ...    ...   
9214            0        1      49      6          4         0       0      0   
9215            1        0     118     10          3         0       0      0   
9216            0        1      52      6          5         0       0      0   
9217            0        1      61      5         49         1       0      1   
9218            0        1     232     22         12         0       0      0   

      indonesia  prabowo  ...  bahas  tertawa  proyek  marah  iman  bangga  \
0             0        0  ...      0        0       0      0     0       0   
1             0        0  ...      0        0       0      0     0       0   
2             0        1  ...      0        0       0      0     0       0   
3             0        0  ...      0        0       0      0     0       0   
4             0        0  ...      0        0       0      0     0       0   
...         ...      ...  ...    ...      ...     ...    ...   ...     ...   
9214          0        0  ...      0        0       0      0     0       0   
9215          0        0  ...      0        0       0      0     0       0   
9216          0        0  ...      0        0       0      0     0       0   
9217          0        0  ...      0        0       0      0     0       0   
9218          0        0  ...      0        0       0      0     0       0   

      muka  paksa  lanjut  bandara  
0        0      0       0        0  
1        0      0       0        0  
2        0      0       0        0  
3        0      0       0        0  
4        0      0       0        0  
...    ...    ...     ...      ...  
9214     0      0       0        0  
9215     0      0       0        0  
9216     0      0       0        0  
9217     0      0       0        0  
9218     0      0       0        0  

[9219 rows x 360 columns]

In [ ]:
Y_train_HS

HS
0      1
1      0
2      1
3      1
4      1
...   ..
9214   1
9215   1
9216   1
9217   1
9218   1

[9219 rows x 1 columns]

# Stratified Train-Test Run

## LogReg

In [ ]:
# all parameters not specified are set to their defaults
lr_HS = LogisticRegression()
lr_HS.fit(X_train_HS, Y_train_HS)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
accuracy_score(Y_test_HS, lr_HS.predict(X_test_HS))

0.7888607594936708

In [ ]:
lr_HS.score(X_stest_HS, Y_stest_HS)

0.7005063291139241

In [ ]:
# all parameters not specified are set to their defaults
lr_HS_all = LogisticRegression()
lr_HS_all.fit(X_train, Y_train_HS)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
accuracy_score(Y_test_HS, lr_HS_all.predict(X_test))

0.7886075949367088

In [ ]:
accuracy_score(Y_stest_HS, lr_HS_all.predict(X_stest))

0.6950379746835443

In [ ]:
# all parameters not specified are set to their defaults
lr_A = LogisticRegression()
lr_A.fit(X_train_A, Y_train_A)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
lr_A.score(X_test_A, Y_test_A)

0.8822784810126583

In [ ]:
# all parameters not specified are set to their defaults
lr_A_all = LogisticRegression()
lr_A_all.fit(X_train, Y_train_A)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
accuracy_score(Y_test_A, lr_A_all.predict(X_test))

0.8850632911392405

## XGBoost

In [ ]:
XGBC_HS = XGBClassifier(
    booster='gbtree',
    eta=0.35,
    gamma=1
)
XGBC_HS.fit(X_train_HS, Y_train_HS)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.35, eval_metric=None,
              feature_types=None, gamma=1, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, ...)

In [ ]:
# make predictions for test data
xgbc_y_pred_hs = XGBC_HS.predict(X_test_HS)
xgbc_hs_pred = [round(value) for value in xgbc_y_pred_hs]

# evaluate predictions
print("Accuracy: %.4f%%" % (accuracy_score(Y_test_HS, xgbc_hs_pred) * 100.0))

Accuracy: 80.7595%


In [ ]:
accuracy_score(Y_test_HS, xgbc_hs_pred)

0.8075949367088607

In [ ]:
print(classification_report(Y_test_HS, xgbc_hs_pred))

              precision    recall  f1-score   support

           0       0.78      0.93      0.85      2282
           1       0.88      0.63      0.74      1668

    accuracy                           0.81      3950
   macro avg       0.83      0.78      0.79      3950
weighted avg       0.82      0.81      0.80      3950



In [ ]:
# evaluate predictions
print("Accuracy: %.4f%%" % (accuracy_score(Y_stest_HS, [round(value) for value in XGBC_HS.predict(X_stest_HS)]) * 100.0))

Accuracy: 71.0127%


In [ ]:
XGBC_HS_all = XGBClassifier(
    booster='gbtree',
    eta=0.35,
    gamma=1
)
XGBC_HS_all.fit(X_train, Y_train_HS)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.35, eval_metric=None,
              feature_types=None, gamma=1, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, ...)

In [ ]:
accuracy_score(Y_test_HS, XGBC_HS_all.predict(X_test))

0.8073417721518987

In [ ]:
XGBC_A = XGBClassifier(
    booster='gbtree',
    eta=0.35,
    gamma=1
)
XGBC_A.fit(X_train_A, Y_train_A)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.35, eval_metric=None,
              feature_types=None, gamma=1, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, ...)

In [ ]:
# make predictions for test data
xgbc_y_pred_a = XGBC_A.predict(X_test_A)
xgbc_a_pred = [round(value) for value in xgbc_y_pred_a]

# evaluate predictions
print("Accuracy: %.4f%%" % (accuracy_score(Y_test_A, xgbc_a_pred) * 100.0))

Accuracy: 88.9873%


In [ ]:
print(classification_report(Y_test_A, xgbc_a_pred))

              precision    recall  f1-score   support

           0       0.88      0.95      0.91      2438
           1       0.91      0.79      0.85      1512

    accuracy                           0.89      3950
   macro avg       0.89      0.87      0.88      3950
weighted avg       0.89      0.89      0.89      3950



In [ ]:
# evaluate predictions
print("Accuracy: %.4f%%" % (accuracy_score(Y_stest_A, [round(value) for value in XGBC_A.predict(X_stest_A)]) * 100.0))

Accuracy: 86.2684%


In [ ]:
print(classification_report(Y_stest_A, [round(value) for value in XGBC_A.predict(X_stest_A)]))

              precision    recall  f1-score   support

           0       0.85      0.95      0.90     12190
           1       0.90      0.72      0.80      7560

    accuracy                           0.86     19750
   macro avg       0.87      0.84      0.85     19750
weighted avg       0.87      0.86      0.86     19750



In [ ]:
XGBC_A_all = XGBClassifier(
    booster='gbtree',
    eta=0.35,
    gamma=1
)
XGBC_A_all.fit(X_train, Y_train_A)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.35, eval_metric=None,
              feature_types=None, gamma=1, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, ...)

In [ ]:
accuracy_score(Y_test_A, XGBC_A_all.predict(X_test))

0.8873417721518987

## CatBoost

In [ ]:
hs_train_dataset = Pool(data=X_train_HS,
                     label=Y_train_HS)
hs_eval_dataset = Pool(data=X_test_HS,
                     label=Y_test_HS)

# Initialize CatBoostClassifier
hs_cbc = CatBoostClassifier(iterations=17,
                           learning_rate=1,
                           depth=2)

hs_cbc.fit(hs_train_dataset, eval_set=hs_eval_dataset)

0:	learn: 0.6604992	test: 0.6471065	best: 0.6471065 (0)	total: 52.9ms	remaining: 847ms
1:	learn: 0.6424937	test: 0.6351506	best: 0.6351506 (1)	total: 63.1ms	remaining: 473ms
2:	learn: 0.6305656	test: 0.6216650	best: 0.6216650 (2)	total: 70.7ms	remaining: 330ms
3:	learn: 0.6202958	test: 0.6111411	best: 0.6111411 (3)	total: 79.3ms	remaining: 258ms
4:	learn: 0.6095936	test: 0.5964405	best: 0.5964405 (4)	total: 87.8ms	remaining: 211ms
5:	learn: 0.6006182	test: 0.5867262	best: 0.5867262 (5)	total: 96.3ms	remaining: 177ms
6:	learn: 0.5938400	test: 0.5760413	best: 0.5760413 (6)	total: 104ms	remaining: 149ms
7:	learn: 0.5887546	test: 0.5694188	best: 0.5694188 (7)	total: 112ms	remaining: 126ms
8:	learn: 0.5845364	test: 0.5674096	best: 0.5674096 (8)	total: 120ms	remaining: 107ms
9:	learn: 0.5801202	test: 0.5611630	best: 0.5611630 (9)	total: 128ms	remaining: 89.6ms
10:	learn: 0.5758278	test: 0.5525202	best: 0.5525202 (10)	total: 137ms	remaining: 74.7ms
11:	learn: 0.5718263	test: 0.5479368	best: 0

In [ ]:
cbc_hs_preds = hs_cbc.predict(X_test_HS) 
accuracy = sum(1 for x,y in zip(Y_test_HS.values, cbc_hs_preds) if (abs(float(x)-float(y)) < 0.5)) / len(Y_test_HS)
accuracy

0.7524050632911392

In [ ]:
accuracy_score(Y_test_HS, hs_cbc.predict(X_test_HS))

0.7524050632911392

In [ ]:
cbc_hs_spreds = hs_cbc.predict(X_stest_HS) 
accuracy = sum(1 for x,y in zip(Y_stest_HS.values, cbc_hs_spreds) if (abs(float(x)-float(y)) < 0.5)) / len(Y_stest_HS)
accuracy

0.6815189873417722

In [ ]:
A_train_dataset = Pool(data=X_train_A,
                     label=Y_train_A)
A_eval_dataset = Pool(data=X_test_A,
                     label=Y_test_A)

# Initialize CatBoostClassifier
a_cbc = CatBoostClassifier(iterations=15,
                           learning_rate=1,
                           depth=2)

a_cbc.fit(A_train_dataset, eval_set=A_eval_dataset)

0:	learn: 0.5877922	test: 0.5164967	best: 0.5164967 (0)	total: 9.25ms	remaining: 129ms
1:	learn: 0.5605352	test: 0.4945186	best: 0.4945186 (1)	total: 17.6ms	remaining: 114ms
2:	learn: 0.5466890	test: 0.4793689	best: 0.4793689 (2)	total: 25.1ms	remaining: 100ms
3:	learn: 0.5354636	test: 0.4641345	best: 0.4641345 (3)	total: 34.1ms	remaining: 93.8ms
4:	learn: 0.5288048	test: 0.4535334	best: 0.4535334 (4)	total: 41ms	remaining: 82ms
5:	learn: 0.5227799	test: 0.4459576	best: 0.4459576 (5)	total: 49ms	remaining: 73.5ms
6:	learn: 0.5172600	test: 0.4347404	best: 0.4347404 (6)	total: 56.7ms	remaining: 64.8ms
7:	learn: 0.5117065	test: 0.4240980	best: 0.4240980 (7)	total: 64.8ms	remaining: 56.7ms
8:	learn: 0.5050641	test: 0.4217437	best: 0.4217437 (8)	total: 75.8ms	remaining: 50.5ms
9:	learn: 0.5005038	test: 0.4154405	best: 0.4154405 (9)	total: 83.2ms	remaining: 41.6ms
10:	learn: 0.4965435	test: 0.4100538	best: 0.4100538 (10)	total: 91.8ms	remaining: 33.4ms
11:	learn: 0.4937636	test: 0.4062970	be

In [ ]:
cbc_a_preds = a_cbc.predict(X_test) 
accuracy = sum(1 for x,y in zip(Y_test_A.values, cbc_a_preds) if (abs(float(x)-float(y)) < 0.5)) / len(Y_test_A)
accuracy

0.8187341772151899

In [ ]:
cbc_a_spreds = a_cbc.predict(X_stest) 
accuracy = sum(1 for x,y in zip(Y_stest_A.values, cbc_a_spreds) if (abs(float(x)-float(y)) < 0.5)) / len(Y_stest_A)
accuracy

0.7931139240506329

In [ ]:
# Initialize CatBoostClassifier
a_cbc_all = CatBoostClassifier(iterations=37,
                           learning_rate=1,
                           depth=2)

a_cbc_all.fit(
    Pool(
        data=X_train,
        label=Y_train_A), 
    eval_set=Pool(
        data=X_test,
        label=Y_test_A)
    )

0:	learn: 0.5758887	test: 0.4980083	best: 0.4980083 (0)	total: 4.74ms	remaining: 171ms
1:	learn: 0.5530866	test: 0.4852933	best: 0.4852933 (1)	total: 9.04ms	remaining: 158ms
2:	learn: 0.5392049	test: 0.4694625	best: 0.4694625 (2)	total: 13.6ms	remaining: 154ms
3:	learn: 0.5329393	test: 0.4572239	best: 0.4572239 (3)	total: 18.6ms	remaining: 154ms
4:	learn: 0.5280907	test: 0.4478044	best: 0.4478044 (4)	total: 26ms	remaining: 166ms
5:	learn: 0.5215470	test: 0.4375809	best: 0.4375809 (5)	total: 31.2ms	remaining: 161ms
6:	learn: 0.5139204	test: 0.4227632	best: 0.4227632 (6)	total: 36.3ms	remaining: 155ms
7:	learn: 0.5114764	test: 0.4192776	best: 0.4192776 (7)	total: 40.9ms	remaining: 148ms
8:	learn: 0.5080511	test: 0.4157195	best: 0.4157195 (8)	total: 45.3ms	remaining: 141ms
9:	learn: 0.5037471	test: 0.4119565	best: 0.4119565 (9)	total: 50.1ms	remaining: 135ms
10:	learn: 0.4972486	test: 0.4063541	best: 0.4063541 (10)	total: 54.8ms	remaining: 129ms
11:	learn: 0.4922777	test: 0.4019935	best: 

In [ ]:
cbc_a_allpreds = a_cbc_all.predict(X_test) 
accuracy = sum(1 for x,y in zip(Y_test_A.values, cbc_a_allpreds) if (abs(float(x)-float(y)) < 0.5)) / len(Y_test_A)
accuracy

0.8612658227848101

## MLP

In [ ]:
parameter_space = {
    'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

In [ ]:
hs_mlpc_model = MLPClassifier(random_state=1, max_iter=100)

In [ ]:
# from sklearn.model_selection import GridSearchCV

# mlpc_hs_gs = GridSearchCV(hs_mlpc_model, parameter_space, n_jobs=-1, cv=3)
# mlpc_hs_gs.fit(X_train_HS, Y_train_HS.HS.values)

In [ ]:
# # Best paramete set
# print('Best parameters found:\n', mlpc_hs_gs.best_params_)

# # All results
# means = mlpc_hs_gs.cv_results_['mean_test_score']
# stds = mlpc_hs_gs.cv_results_['std_test_score']
# for mean, std, params in zip(means, stds, mlpc_hs_gs.cv_results_['params']):
#     print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

In [ ]:
# Best parameters found:
#  {'activation': 'relu', 'alpha': 0.05, 'hidden_layer_sizes': (50, 100, 50), 'learning_rate': 'constant', 'solver': 'adam'}

hs_mlpc_tunedmodel = MLPClassifier(
    random_state=1, 
    max_iter=100,
    activation='relu',
    alpha=0.05,
    hidden_layer_sizes=(50, 100, 50),
    learning_rate='constant',
    solver='adam'
    )
hs_mlpc_tunedmodel.fit(X_train_HS, Y_train_HS.HS.values)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(alpha=0.05, hidden_layer_sizes=(50, 100, 50), max_iter=100,
              random_state=1)

In [ ]:
hs_mlpc_tunedmodel.score(X_test_HS, Y_test_HS.HS.values)

0.7648101265822784

In [ ]:
hs_mlpc = MLPClassifier(random_state=1, max_iter=100)
hs_mlpc.fit(X_train_HS, Y_train_HS.HS.values)
hs_mlpc.score(X_test_HS, Y_test_HS.HS.values)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


0.7931645569620254

In [ ]:
hs_mlpc_tunedmodel.score(X_stest_HS, Y_stest_HS)

0.685367088607595

In [ ]:
# A_mlpc_model = MLPClassifier(random_state=1, max_iter=100)

# mlpc_A_gs = GridSearchCV(A_mlpc_model, parameter_space, n_jobs=-1, cv=3)
# mlpc_A_gs.fit(X_train_A, Y_train_A.A.values)

In [ ]:
# # Best paramete set
# print('Best parameters found:\n', mlpc_A_gs.best_params_)

# # All results
# means = mlpc_A_gs.cv_results_['mean_test_score']
# stds = mlpc_A_gs.cv_results_['std_test_score']
# for mean, std, params in zip(means, stds, mlpc_A_gs.cv_results_['params']):
#     print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

In [ ]:
# Best parameters found:
#  {'activation': 'relu', 'alpha': 0.05, 'hidden_layer_sizes': (50, 50, 50), 'learning_rate': 'constant', 'solver': 'adam'}

A_mlpc_tunedmodel = MLPClassifier(
    random_state=1, 
    max_iter=100,
    activation='relu',
    alpha=0.05,
    hidden_layer_sizes=(50, 50, 50),
    learning_rate='constant',
    solver='adam'
    )
A_mlpc_tunedmodel.fit(X_train_A, Y_train_A)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1098: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(alpha=0.05, hidden_layer_sizes=(50, 50, 50), max_iter=100,
              random_state=1)

In [ ]:
A_mlpc_tunedmodel.score(X_test_A, Y_test_A)

0.8891139240506329

In [ ]:
A_mlpc_tunedmodel.score(X_stest_A, Y_stest_A)

0.8616202531645569

In [ ]:
a_mlpc = MLPClassifier(random_state=1, max_iter=100)
a_mlpc.fit(X_train_A, Y_train_A.A.values)
a_mlpc.score(X_test_A, Y_test_A.A.values)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


0.8686075949367089

In [ ]:
a_mlpc.get_params()

{'activation': 'relu',
 'alpha': 0.0001,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (100,),
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_fun': 15000,
 'max_iter': 100,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': 1,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

## SVM

### LSVM

In [ ]:
hs_lsvc = LinearSVC()
hs_lsvc.fit(X_train_HS, Y_train_HS.HS.values)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC()

In [ ]:
# print prediction results
lsvc_hs_pred = hs_lsvc.predict(X_test_HS)
print(classification_report(Y_test_HS, lsvc_hs_pred))
# evaluate predictions
print("Accuracy: %.4f%%" % (accuracy_score(Y_test_HS, [round(value) for value in hs_lsvc.predict(X_test_HS)]) * 100.0))

              precision    recall  f1-score   support

           0       0.63      0.99      0.77      2282
           1       0.93      0.19      0.31      1668

    accuracy                           0.65      3950
   macro avg       0.78      0.59      0.54      3950
weighted avg       0.75      0.65      0.58      3950

Accuracy: 65.1392%


In [ ]:
a_lsvc = LinearSVC()
a_lsvc.fit(X_train_A, Y_train_A.A.values)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC()

In [ ]:
# print prediction results
lsvc_a_pred = a_lsvc.predict(X_test_A)
print(classification_report(Y_test_A, lsvc_a_pred))
# evaluate predictions
print("Accuracy: %.4f%%" % (accuracy_score(Y_test_A, [round(value) for value in a_lsvc.predict(X_test_A)]) * 100.0))

              precision    recall  f1-score   support

           0       0.88      0.93      0.90      2438
           1       0.88      0.79      0.83      1512

    accuracy                           0.88      3950
   macro avg       0.88      0.86      0.87      3950
weighted avg       0.88      0.88      0.88      3950

Accuracy: 87.7722%


### SVM

In [ ]:
hs_svm = SVC()
hs_svm.fit(X_train_HS, Y_train_HS.HS.values)

SVC()

In [ ]:
# print prediction results
hs_svm_pred = hs_svm.predict(X_test_HS)
print(classification_report(Y_test_HS, hs_svm_pred))
# evaluate predictions
print("Accuracy: %.4f%%" % (accuracy_score(Y_test_HS, [round(value) for value in hs_svm.predict(X_test_HS)]) * 100.0))

              precision    recall  f1-score   support

           0       0.58      0.98      0.73      2282
           1       0.58      0.04      0.07      1668

    accuracy                           0.58      3950
   macro avg       0.58      0.51      0.40      3950
weighted avg       0.58      0.58      0.45      3950

Accuracy: 58.2025%


In [ ]:
A_svm = SVC(kernel='sigmoid')
A_svm.fit(X_train_A, Y_train_A.A.values)

SVC(kernel='sigmoid')

In [ ]:
# print prediction results
a_svm_pred = A_svm.predict(X_test_A)
print(classification_report(Y_test_A, a_svm_pred))
# evaluate predictions
print("Accuracy: %.4f%%" % (accuracy_score(Y_test_A, [round(value) for value in A_svm.predict(X_test_A)]) * 100.0))

              precision    recall  f1-score   support

           0       0.62      0.61      0.62      2438
           1       0.38      0.38      0.38      1512

    accuracy                           0.53      3950
   macro avg       0.50      0.50      0.50      3950
weighted avg       0.53      0.53      0.53      3950

Accuracy: 52.5823%


In [ ]:
# from sklearn.model_selection import GridSearchCV
  
# # defining parameter range
# param_grid = {'C': [0.1, 1, 10, 100, 1000], 
#               'gamma': [1, 0.1, 0.01],
#               'kernel': ['rbf']} 
  
# grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
  
# # fitting the model for grid search
# grid.fit(X_train_HS, Y_train_HS.HS.values)

Fitting 5 folds for each of 15 candidates, totalling 75 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.578 total time=  41.4s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.578 total time=  22.1s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.578 total time=  20.5s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.578 total time=  22.1s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.578 total time=  21.8s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.578 total time=  25.1s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.578 total time=  23.2s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.578 total time=  21.1s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.578 total time=  22.7s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.578 total time=  21.5s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.568 total time=  18.4s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf;

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01],
                         'kernel': ['rbf']},
             verbose=3)

In [ ]:
# # Best paramete set
print('Best parameters found:\n', grid.best_params_)

Best parameters found:
 {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}


# Bootstrap Sampling

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# https://insidelearningmachines.com/implement-the-bootstrap-method-in-python/

def generate_bootstrap_sample(source_df):
  bwr = source_df.copy()[0:0]
  idx = source_df.index.values
  sample_ids = np.random.choice(idx,replace=True,size=len(idx))

  for i in sample_ids:
    row = source_df.loc[source_df.index == i]
    bwr = bwr.append(row)

  return bwr

In [ ]:
idx = [i for i in range(10)]
ids = np.random.choice(idx,replace=True,size=10)
print(idx)
print(ids)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[6 1 9 1 1 8 0 8 5 8]


In [ ]:
ex_df = train_data.copy()[0:0]
for i in ids:
  row = train_data.loc[train_data.index == i]
  ex_df = ex_df.append(row)

ex_df

Exclamation  Abusive  Length  Words  Uppercase  All Caps  jokowi  islam  \
6            0        0     181     12         15         0       0      0   
1            0        0      77      6          4         0       0      1   
9            0        1     189     15          0         0       0      0   
1            0        0      77      6          4         0       0      1   
1            0        0      77      6          4         0       0      1   
8            0        0      47      5          9         0       0      0   
0            1        1      74      6          2         0       0      0   
8            0        0      47      5          9         0       0      0   
5            1        1     114      7          6         0       0      0   
8            0        0      47      5          9         0       0      0   

   indonesia  prabowo  ...  foto  proyek  marah  iman  bangga  muka  paksa  \
6          0        0  ...     0       0      0     0       0     0      0   
1          0        0  ...     0       0      0     0       0     0      0   
9          0        0  ...     0       0      0     0       0     0      0   
1          0        0  ...     0       0      0     0       0     0      0   
1          0        0  ...     0       0      0     0       0     0      0   
8          0        0  ...     0       0      0     0       0     0      0   
0          0        0  ...     0       0      0     0       0     0      0   
8          0        0  ...     0       0      0     0       0     0      0   
5          0        0  ...     0       0      0     0       0     0      0   
8          0        0  ...     0       0      0     0       0     0      0   

   lanjut  bandara  menteri  
6       0        0        0  
1       0        0        0  
9       0        0        0  
1       0        0        0  
1       0        0        0  
8       0        1        0  
0       0        0        0  
8       0        1        0  
5       0        0        0  
8       0        1        0  

[10 rows x 374 columns]

## run

In [ ]:
bwr_list = []

lr_acc_list_hs = []
xgb_acc_list_hs = []
cat_acc_list_hs = []
mlp_acc_list_hs = []
svm_acc_list_hs = []

lr_acc_list_a = []
xgb_acc_list_a = []
cat_acc_list_a = []
mlp_acc_list_a = []
svm_acc_list_a = []

In [ ]:
# for n in range(10):
#   nbwr = generate_bootstrap_sample(all_train_data)
#   bwr_list.append(nbwr)

#   # ===========
#   # hate speech
#   # ===========
#   xhs, yhs = get_x_y(nbwr, 'all HS')
#   print(len(xhs))

#   hs_lr_model = LogisticRegression().fit(xhs, yhs)
#   lr_acc_list_hs.append(accuracy_score(Y_test_HS, hs_lr_model.predict(X_test)))

#   hs_xgb_model = XGBClassifier(
#                             booster='gbtree',
#                             eta=0.35,
#                             gamma=1
#                             ).fit(xhs, yhs)
#   xgb_acc_list_hs.append(accuracy_score(Y_test_HS, hs_xgb_model.predict(X_test)))

#   hs_cbc_model = CatBoostClassifier(
#                             iterations=17,
#                             learning_rate=1,
#                             depth=2
#                             ).fit(
#                                 Pool(data=xhs, label=yhs), 
#                                 eval_set=Pool(data=X_test, label=Y_test_HS)
#                                 )
#   cat_acc_list_hs.append(accuracy_score(Y_test_HS, hs_cbc_model.predict(X_test)))

#   hs_mlp_model = MLPClassifier(random_state=1, max_iter=100).fit(xhs, yhs.HS.values)
#   mlp_acc_list_hs.append(accuracy_score(Y_test_HS, hs_mlp_model.predict(X_test)))

#   hs_lsvc_model = LinearSVC().fit(xhs, yhs.HS.values)
#   svm_acc_list_hs.append(accuracy_score(Y_test_HS, hs_lsvc_model.predict(X_test)))

#   # ===========
#   # abusive
#   # ===========
#   xa, ya = get_x_y(nbwr, 'all A')
#   print(len(xa))

#   a_lr_model = LogisticRegression().fit(xa, ya)
#   lr_acc_list_a.append(accuracy_score(Y_test_A, a_lr_model.predict(X_test)))

#   a_xgb_model = XGBClassifier(
#                             booster='gbtree',
#                             eta=0.35,
#                             gamma=1
#                             ).fit(xa, ya)
#   xgb_acc_list_a.append(accuracy_score(Y_test_A, a_xgb_model.predict(X_test)))

#   a_cbc_model = CatBoostClassifier(
#                             iterations=15,
#                             learning_rate=1,
#                             depth=2
#                             ).fit(
#                                 Pool(data=xa, label=ya), 
#                                 eval_set=Pool(data=X_test, label=Y_test_A)
#                                 )
#   cat_acc_list_a.append(accuracy_score(Y_test_A, a_cbc_model.predict(X_test)))

#   a_mlp_model = MLPClassifier(random_state=1, max_iter=100).fit(xa, ya.A.values)
#   mlp_acc_list_a.append(accuracy_score(Y_test_A, a_mlp_model.predict(X_test)))

#   a_lsvc_model = LinearSVC().fit(xa, ya.A.values)
#   svm_acc_list_a.append(accuracy_score(Y_test_A, a_lsvc_model.predict(X_test)))


In [ ]:
for n in range(30):
    nbwr = generate_bootstrap_sample(all_train_data)
    bwr_list.append(nbwr)

    # ===========
    # hate speech
    # ===========
    xhs, yhs = get_x_y(nbwr, 'HS')
    print(len(xhs))

    hs_lr_model = LogisticRegression().fit(xhs, yhs)
    lr_acc_list_hs.append(accuracy_score(Y_test_HS, hs_lr_model.predict(X_test_HS)))

    hs_xgb_model = XGBClassifier(
                              booster='gbtree',
                              eta=0.35,
                              gamma=1
                              ).fit(xhs, yhs)
    xgb_acc_list_hs.append(accuracy_score(Y_test_HS, hs_xgb_model.predict(X_test_HS)))

    hs_cbc_model = CatBoostClassifier(
                              iterations=17,
                              learning_rate=1,
                              depth=2
                              ).fit(
                                  Pool(data=xhs, label=yhs), 
                                  eval_set=Pool(data=X_test_HS, label=Y_test_HS)
                                  )
    cat_acc_list_hs.append(accuracy_score(Y_test_HS, hs_cbc_model.predict(X_test_HS)))

    hs_mlp_model = MLPClassifier(random_state=1, max_iter=100).fit(xhs, yhs.HS.values)
    mlp_acc_list_hs.append(accuracy_score(Y_test_HS, hs_mlp_model.predict(X_test_HS)))

    hs_lsvc_model = LinearSVC().fit(xhs, yhs.HS.values)
    svm_acc_list_hs.append(accuracy_score(Y_test_HS, hs_lsvc_model.predict(X_test_HS)))

    # ===========
    # abusive
    # ===========
    xa, ya = get_x_y(nbwr, 'A')
    print("!!!!!!!!!!!!!!!!", len(xa))

    a_lr_model = LogisticRegression().fit(xa, ya)
    lr_acc_list_a.append(accuracy_score(Y_test_A, a_lr_model.predict(X_test_A)))

    a_xgb_model = XGBClassifier(
                              booster='gbtree',
                              eta=0.35,
                              gamma=1
                              ).fit(xa, ya)
    xgb_acc_list_a.append(accuracy_score(Y_test_A, a_xgb_model.predict(X_test_A)))

    a_cbc_model = CatBoostClassifier(
                              iterations=15,
                              learning_rate=1,
                              depth=2
                              ).fit(
                                  Pool(data=xa, label=ya), 
                                  eval_set=Pool(data=X_test_A, label=Y_test_A)
                                  )
    cat_acc_list_a.append(accuracy_score(Y_test_A, a_cbc_model.predict(X_test_A)))

    a_mlp_model = MLPClassifier(random_state=1, max_iter=100).fit(xa, ya.A.values)
    mlp_acc_list_a.append(accuracy_score(Y_test_A, a_mlp_model.predict(X_test_A)))

    a_lsvc_model = LinearSVC().fit(xa, ya.A.values)
    svm_acc_list_a.append(accuracy_score(Y_test_A, a_lsvc_model.predict(X_test_A)))

9219


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0:	learn: 0.6570183	test: 0.6613834	best: 0.6613834 (0)	total: 50.6ms	remaining: 809ms
1:	learn: 0.6396900	test: 0.6356174	best: 0.6356174 (1)	total: 54.7ms	remaining: 410ms
2:	learn: 0.6301977	test: 0.6223479	best: 0.6223479 (2)	total: 58.8ms	remaining: 274ms
3:	learn: 0.6185500	test: 0.6107423	best: 0.6107423 (3)	total: 62.9ms	remaining: 205ms
4:	learn: 0.6063960	test: 0.6011337	best: 0.6011337 (4)	total: 66.9ms	remaining: 161ms
5:	learn: 0.5970977	test: 0.5912019	best: 0.5912019 (5)	total: 70.8ms	remaining: 130ms
6:	learn: 0.5892099	test: 0.5810458	best: 0.5810458 (6)	total: 74.8ms	remaining: 107ms
7:	learn: 0.5832831	test: 0.5778153	best: 0.5778153 (7)	total: 78.7ms	remaining: 88.5ms
8:	learn: 0.5775490	test: 0.5713594	best: 0.5713594 (8)	total: 82.6ms	remaining: 73.4ms
9:	learn: 0.5734987	test: 0.5642903	best: 0.5642903 (9)	total: 86.4ms	remaining: 60.5ms
10:	learn: 0.5700450	test: 0.5611032	best: 0.5611032 (10)	total: 90.2ms	remaining: 49.2ms
11:	learn: 0.5659171	test: 0.5547631	

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


!!!!!!!!!!!!!!!! 9219


...

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


!!!!!!!!!!!!!!!! 9219


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0:	learn: 0.5732707	test: 0.5133095	best: 0.5133095 (0)	total: 3.95ms	remaining: 55.3ms
1:	learn: 0.5469973	test: 0.4916486	best: 0.4916486 (1)	total: 8.12ms	remaining: 52.8ms
2:	learn: 0.5362310	test: 0.4836734	best: 0.4836734 (2)	total: 12.2ms	remaining: 48.9ms
3:	learn: 0.5246716	test: 0.4666973	best: 0.4666973 (3)	total: 15.9ms	remaining: 43.8ms
4:	learn: 0.5149198	test: 0.4495285	best: 0.4495285 (4)	total: 20ms	remaining: 39.9ms
5:	learn: 0.5075687	test: 0.4390885	best: 0.4390885 (5)	total: 23.8ms	remaining: 35.6ms
6:	learn: 0.5022349	test: 0.4279900	best: 0.4279900 (6)	total: 27.7ms	remaining: 31.6ms
7:	learn: 0.4964321	test: 0.4227731	best: 0.4227731 (7)	total: 31.9ms	remaining: 27.9ms
8:	learn: 0.4893943	test: 0.4196247	best: 0.4196247 (8)	total: 35.8ms	remaining: 23.9ms
9:	learn: 0.4855939	test: 0.4167028	best: 0.4167028 (9)	total: 39.8ms	remaining: 19.9ms
10:	learn: 0.4822233	test: 0.4108023	best: 0.4108023 (10)	total: 43.5ms	remaining: 15.8ms
11:	learn: 0.4782504	test: 0.405

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


## result

In [ ]:

print(lr_acc_list_hs)
print(xgb_acc_list_hs)
print(cat_acc_list_hs)
print(mlp_acc_list_hs)
print(svm_acc_list_hs)
print()
print(lr_acc_list_a)
print(xgb_acc_list_a)
print(cat_acc_list_a)
print(mlp_acc_list_a)
print(svm_acc_list_a)

[0.7764556962025316, 0.7853164556962026, 0.7812658227848102, 0.7860759493670886, 0.7764556962025316, 0.779746835443038, 0.7875949367088607, 0.7787341772151899, 0.7721518987341772, 0.7746835443037975, 0.7772151898734178, 0.7772151898734178, 0.7792405063291139, 0.7774683544303798, 0.7784810126582279, 0.7718987341772152, 0.7888607594936708, 0.7865822784810127, 0.789873417721519, 0.7716455696202532, 0.7762025316455696, 0.780506329113924, 0.769873417721519, 0.7749367088607595, 0.7721518987341772, 0.7792405063291139, 0.7772151898734178, 0.7769620253164558, 0.7762025316455696, 0.7853164556962026]
[0.7886075949367088, 0.7908860759493671, 0.7949367088607595, 0.8017721518987342, 0.789620253164557, 0.7881012658227848, 0.790126582278481, 0.7934177215189874, 0.7848101265822784, 0.7850632911392406, 0.7934177215189874, 0.7835443037974683, 0.7956962025316455, 0.7992405063291139, 0.7974683544303798, 0.7949367088607595, 0.8, 0.7865822784810127, 0.7924050632911392, 0.7812658227848102, 0.7924050632911392,

In [ ]:
print("lr hs: ", sum(lr_acc_list_hs)/len(lr_acc_list_hs))
print("xgbc hs: ", sum(xgb_acc_list_hs)/len(xgb_acc_list_hs))
print("cbc hs: ", sum(cat_acc_list_hs)/len(cat_acc_list_hs))
print("mlp hs: ", sum(mlp_acc_list_hs)/len(mlp_acc_list_hs))
print("lsvm hs: ", sum(svm_acc_list_hs)/len(svm_acc_list_hs))
print()
print("lr a: ", sum(lr_acc_list_a)/len(lr_acc_list_a))
print("xgbc a: ", sum(xgb_acc_list_a)/len(xgb_acc_list_a))
print("cbc a: ", sum(cat_acc_list_a)/len(cat_acc_list_a))
print("mlp a: ", sum(mlp_acc_list_a)/len(mlp_acc_list_a))
print("lsvm a: ", sum(svm_acc_list_a)/len(svm_acc_list_a))

lr hs:  0.7788523206751055
xgbc hs:  0.7906413502109706
cbc hs:  0.7444725738396623
mlp hs:  0.767518987341772
lsvm hs:  0.6591814345991561

lr a:  0.8748607594936709
xgbc a:  0.876278481012658
cbc a:  0.8131223628691981
mlp a:  0.8587594936708862
lsvm a:  0.7951645569620255


In [ ]:
print("lr hs: ", max(lr_acc_list_hs))
print("xgbc hs: ", max(xgb_acc_list_hs))
print("cbc hs: ", max(cat_acc_list_hs))
print("mlp hs: ", max(mlp_acc_list_hs))
print("lsvm hs: ", max(svm_acc_list_hs))
print()
print("lr a: ", max(lr_acc_list_a))
print("xgbc a: ", max(xgb_acc_list_a))
print("cbc a: ", max(cat_acc_list_a))
print("mlp a: ", max(mlp_acc_list_a))
print("lsvm a: ", max(svm_acc_list_a))

lr hs:  0.789873417721519
xgbc hs:  0.8017721518987342
cbc hs:  0.7622784810126583
mlp hs:  0.7848101265822784
lsvm hs:  0.7650632911392405

lr a:  0.8817721518987341
xgbc a:  0.8817721518987341
cbc a:  0.8440506329113924
mlp a:  0.8726582278481013
lsvm a:  0.8734177215189873


In [ ]:
print("lr hs: ", min(lr_acc_list_hs))
print("xgbc hs: ", min(xgb_acc_list_hs))
print("cbc hs: ", min(cat_acc_list_hs))
print("mlp hs: ", min(mlp_acc_list_hs))
print("lsvm hs: ", min(svm_acc_list_hs))
print()
print("lr a: ", min(lr_acc_list_a))
print("xgbc a: ", min(xgb_acc_list_a))
print("cbc a: ", min(cat_acc_list_a))
print("mlp a: ", min(mlp_acc_list_a))
print("lsvm a: ", min(svm_acc_list_a))

lr hs:  0.769873417721519
xgbc hs:  0.7812658227848102
cbc hs:  0.7260759493670886
mlp hs:  0.7427848101265823
lsvm hs:  0.46455696202531643

lr a:  0.8660759493670886
xgbc a:  0.8683544303797468
cbc a:  0.769113924050633
mlp a:  0.8384810126582278
lsvm a:  0.44734177215189874


# Paired t-Test

In [ ]:
print("LogReg | XGBClassifier")
print("     => ", ttest_rel(lr_acc_list_hs, xgb_acc_list_hs))
print("LogReg | CatBoostClassifier")
print("     => ", ttest_rel(lr_acc_list_hs, cat_acc_list_hs))
print("LogReg | MLP")
print("     => ", ttest_rel(lr_acc_list_hs, mlp_acc_list_hs))
print("LogReg | SVM")
print("     => ", ttest_rel(lr_acc_list_hs, svm_acc_list_hs))
print("XGBClassifier | CatBoostClassifier")
print("     => ", ttest_rel(xgb_acc_list_hs, cat_acc_list_hs))
print("XGBClassifier | MLP")model
print("     => ", ttest_rel(xgb_acc_list_hs, mlp_acc_list_hs))
print("XGBClassifier | SVM")
print("     => ", ttest_rel(xgb_acc_list_hs, svm_acc_list_hs))
print("CatBoostClassifier | MLP") 
print("     => ", ttest_rel(cat_acc_list_hs, mlp_acc_list_hs))
print("CatBoostClassifier | SVM")
print("     => ", ttest_rel(cat_acc_list_hs, svm_acc_list_hs))
print("MLP | SVM")
print("     => ", ttest_rel(mlp_acc_list_hs, svm_acc_list_hs))

LogReg | XGBClassifier
     =>  TtestResult(statistic=-10.229267650139604, pvalue=3.9364378785581636e-11, df=29)
LogReg | CatBoostClassifier
     =>  TtestResult(statistic=22.2967247695774, pvalue=8.457658289108301e-20, df=29)
LogReg | MLP
     =>  TtestResult(statistic=4.921167448433303, pvalue=3.158020497705495e-05, df=29)
LogReg | SVM
     =>  TtestResult(statistic=7.984750814121228, pvalue=8.327890704684079e-09, df=29)
XGBClassifier | CatBoostClassifier
     =>  TtestResult(statistic=30.760520762193355, pvalue=1.0863630752569656e-23, df=29)
XGBClassifier | MLP
     =>  TtestResult(statistic=11.229000916114522, pvalue=4.467536621555769e-12, df=29)
XGBClassifier | SVM
     =>  TtestResult(statistic=8.68950085044461, pvalue=1.4437107713602575e-09, df=29)
CatBoostClassifier | MLP
     =>  TtestResult(statistic=-10.600953509076211, pvalue=1.7275680841127383e-11, df=29)
CatBoostClassifier | SVM
     =>  TtestResult(statistic=5.773172688902341, pvalue=2.9727739507794506e-06, df=29)
MLP | 

In [ ]:
print("LogReg | XGBClassifier", ttest_rel(lr_acc_list_a, xgb_acc_list_a))
print("LogReg | CatBoostClassifier", ttest_rel(lr_acc_list_a, cat_acc_list_a))
print("LogReg | MLP", ttest_rel(lr_acc_list_a, mlp_acc_list_a))
print("LogReg | SVM", ttest_rel(lr_acc_list_a, svm_acc_list_a))
print("XGBClassifier | CatBoostClassifier", ttest_rel(xgb_acc_list_a, cat_acc_list_a))
print("XGBClassifier | MLP", ttest_rel(xgb_acc_list_a, mlp_acc_list_a))
print("XGBClassifier | SVM", ttest_rel(xgb_acc_list_a, svm_acc_list_a))
print("CatBoostClassifier | MLP", ttest_rel(cat_acc_list_a, mlp_acc_list_a))
print("CatBoostClassifier | SVM", ttest_rel(cat_acc_list_a, svm_acc_list_a))
print("MLP | SVM", ttest_rel(mlp_acc_list_a, svm_acc_list_a))

LogReg | XGBClassifier TtestResult(statistic=-1.563339497854681, pvalue=0.12882084786742498, df=29)
LogReg | CatBoostClassifier TtestResult(statistic=24.04480478756136, pvalue=1.0565184921868412e-20, df=29)
LogReg | MLP TtestResult(statistic=11.972864566606255, pvalue=9.5727880894555e-13, df=29)
LogReg | SVM TtestResult(statistic=4.382103706895513, pvalue=0.00014057525695661472, df=29)
XGBClassifier | CatBoostClassifier TtestResult(statistic=23.594494319251833, pvalue=1.781461296128588e-20, df=29)
XGBClassifier | MLP TtestResult(statistic=13.889001267222858, pvalue=2.402665672703015e-14, df=29)
XGBClassifier | SVM TtestResult(statistic=4.490156581819347, pvalue=0.00010433445419872066, df=29)
CatBoostClassifier | MLP TtestResult(statistic=-14.618469860513063, pvalue=6.516529218943259e-15, df=29)
CatBoostClassifier | SVM TtestResult(statistic=0.9903579207836073, pvalue=0.3301910002576467, df=29)
MLP | SVM TtestResult(statistic=3.441065162974724, pvalue=0.0017791788694304725, df=29)
